In [44]:
import tushare as ts
from sklearn.model_selection import train_test_split
import datetime
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
today = int(datetime.datetime.now().strftime("%Y%m%d"))

In [45]:
ts.set_token('4f2c81a89e1a43100569f800f658a0a26f55ba57c93908691c17b681')
pro = ts.pro_api()

交易所代码 CFFEX-中金所 DCE-大商所 CZCE-郑商所 SHFE-上期所 INE-上海国际能源交易中心 GFEX-广州期货交易所

In [46]:
# 拉取数据
df = pro.fut_daily(**{
    "trade_date": "",
    # "ts_code": "AG2307.SHF",
    # "ts_code": "OI2309.ZCE",
    # "ts_code": "MA2309.ZCE",
    "ts_code": "V2309.DCE",
    "exchange": "DCE",
    "start_date": today-10000,
    "end_date": today,
    "limit": 10000,
    "offset": "",
}, fields=[
    "ts_code",
    "trade_date",
    "pre_close",
    "pre_settle",
    "open",
    "high",
    "low",
    "close",
    "settle",
    "change1",
    "change2",
    "vol",
    "amount",
    "oi",
    "oi_chg",
    "delv_settle"
])
df.insert(0, 'number', range(1, len(df)+1))

In [47]:
df.to_csv('futures_daily.csv', index=False)

In [48]:
# 读入数据
# df = pd.read_csv('futures.csv')

# 构建特征矩阵
features = ['change1', 'open', 'close', 'oi', 'vol']
X = np.array(df[features])
y = np.array(df['change1'] > 0)

# 划分训练集和测试集
train_size = int(0.7 * len(df))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# 建立随机森林模型
rfc = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
rfc.fit(X_train, y_train)

# 预测测试集
y_pred = rfc.predict(X_test)

# 评估模型准确度
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)
# 假设你已经得到了一个新的数据点，保存在一个名为new_data的DataFrame中
new_data = pd.DataFrame({'change1': [84], 
                         'open': [4777], 
                         'close': [4776], 
                         'oi': [53], 
                         'vol': [824]})

# 对新数据进行预测
y_pred = rfc.predict_proba(new_data)

# 输出涨跌概率
print('下跌概率:', y_pred[:, 0])
print('上涨概率:', y_pred[:, 1])

Accuracy: 1.0
下跌概率: [0.07733333]
上涨概率: [0.92266667]


c:\Users\DuYih\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [49]:
# 加载数据
# data = pd.read_csv('futures.csv')
data=df

# 构建特征矩阵
features = ['open', 'close', 'high', 'low']
X = []
for i in range(3, len(data)):
    x_i = data.loc[i-3:i-1, features].values.reshape(-1)
    X.append(x_i)
X = np.array(X)

# 标记目标变量
y = np.where(data['change1'][3:] > 0, 1, 0)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 训练模型
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train, y_train)

# 评估模型
y_pred = rfc.predict(X_test)
print('准确率：', accuracy_score(y_test, y_pred))
print('精确率：', precision_score(y_test, y_pred))
print('召回率：', recall_score(y_test, y_pred))
# 构造新数据
new_data = data.loc[len(data)-3:len(data)-1, features].values.reshape(-1, 12)

# 预测涨跌概率
y_pred = rfc.predict_proba(new_data)

# 输出涨跌概率
print('下跌概率:', y_pred[0, 0])
print('上涨概率:', y_pred[0, 1])

准确率： 0.6785714285714286
精确率： 0.7
召回率： 0.5384615384615384
下跌概率: 0.48
上涨概率: 0.52


In [50]:
# 加载数据
# data = pd.read_csv('futures_daily.csv')
data = df

# 构建特征矩阵
features = ['open', 'close', 'high', 'low']
X = []
for i in range(3, len(data)):
    x_i = data.loc[i-3:i-1, features].values.reshape(-1)
    X.append(x_i)
X = np.array(X)

# 标记目标变量
y = np.where(data['change1'][3:] > 0, 1, 0)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 训练模型
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train, y_train)

# 评估模型
y_pred = rfc.predict(X_test)
print('准确率：', accuracy_score(y_test, y_pred))
print('精确率：', precision_score(y_test, y_pred))
print('召回率：', recall_score(y_test, y_pred))

# 构造新数据
new_features = ['open', 'close', 'high', 'low']
new_X = []
for i in range(len(data)-3, len(data)):
    x_i = data.loc[i-3:i-1, new_features].values.reshape(-1)
    new_X.append(x_i)
new_X = np.array(new_X)

# 标记目标变量
new_y = np.array([data['change1'][len(data)-4]])

# 预测涨跌概率
new_y_pred = rfc.predict_proba(new_X)

# 输出涨跌概率
print('下跌概率:', new_y_pred[0, 0])
print('上涨概率:', new_y_pred[0, 1])


准确率： 0.6785714285714286
精确率： 0.7
召回率： 0.5384615384615384
下跌概率: 0.76
上涨概率: 0.24


In [51]:
print(new_X)

[[6111. 6020. 6135. 6009. 6010. 6105. 6128. 5965. 6075. 6015. 6075. 5979.]
 [6010. 6105. 6128. 5965. 6075. 6015. 6075. 5979. 6092. 6069. 6130. 6058.]
 [6075. 6015. 6075. 5979. 6092. 6069. 6130. 6058. 6178. 6092. 6217. 6088.]]
